<a href="https://colab.research.google.com/github/k1151msarandega/QuCode-21-Days-of-Quantum-Challenge-Diary/blob/main/Day05_Linear_Algebra_for_Quantum_Computing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 05 — Linear Algebra for Quantum Computing

> _QuCode 21 Days of Quantum Challenge — Learning notebook_
>
> **Date:** 2025-09-05  
> **Author:** Kudzai Musarandega  
> **Tags:** quantum, learning, challenge, day-05
>
> **Learning objectives**
> - Build composite state spaces via **tensor products**; reason about dimensions and bases.
> - Compute **inner** and **outer** products; use orthonormal bases and Kronecker deltas.
> - Recognise **product** vs **entangled** states (Schmidt rank viewpoint).
> - Work with **operators** on subsystems: $A\otimes I$, $I\otimes B$, and multi-qubit gates.
> - Verify and use **unitary** matrices: $U^\dagger U=I$, eigenvalues on the unit circle, inner-product preservation.
>
>
> **Key takeaways (summary-first)**
> - Composite Hilbert space: $\mathcal H=\mathcal H_1\otimes\mathcal H_2$ with $\dim\mathcal H=\dim\mathcal H_1\cdot\dim\mathcal H_2$; basis $|u_i\rangle\otimes|v_j\rangle$.
> - Inner product axioms (complex): linear in the **right** slot, conjugate-symmetric, positive-definite. For product > kets,  
  $$\langle a\!\otimes\!b\,|\,c\!\otimes\!d\rangle=\langle a|c\rangle\,\langle b|d\rangle.$$
> - Outer product $|\psi\rangle\langle\phi|$ is a rank-1 operator (projector if $\psi=\phi$ and $\|\psi\|=1$).
> - **Entanglement:** a 2-qubit state is entangled iff its coefficient matrix has rank $>1$ (non-factorisable).
> - **Unitary** $U$: $U^\dagger U=I$; preserves norms/angles/probabilities; eigenvalues $e^{i\theta}$ (unit modulus).



## Resources
- **Official/Assigned:**
    - [Tensor product state spaces - Professor M does Science](https://youtu.be/kz3206S2B6Q?si=gAuNKNnC51Ux-VZc)
    - [What is an inner product? | Maths of Quantum Mechanics - Quantum Sense](https://youtu.be/3N2vN76E-QA?si=1F1cJ1ZEjTUN7ynQ)
    - [What are unitary operators? | Maths of Quantum Mechanics - Quantum Sense](https://youtu.be/dD-oYfhSKhg?si=bqtdARfOtXv9X6di)
- **Extra reading:**
    - [Unitary Transforms (Quantum Mechanics and Computation) - Wobbly Bit](https://youtu.be/PUBE_x2ikhM?si=M7tWJbnvuINdduD4)
    - [Unitary Matrix | Quantum Computing - RAFEEK KHAN](https://youtu.be/TUWOnNf2NGg?si=z3TeDl4SJFJFhit1)
- **Original notes:**


In [ ]:
# %% [markdown]
# ### Environment setup (Colab)
# If you are running on Colab for the first time today, uncomment to install.
# This cell intentionally avoids heavy installs by default.
#
# !pip -q install qiskit pennylane matplotlib numpy

import sys, platform, math, json, numpy as np

print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
np.random.seed(42)


## 1. Concepts in brief
- Tensor product glues smaller vector spaces into a bigger one: bases multiply, dimensions multiply.
- Inner product generalises the dot product: sesquilinear (R-linear / L-antilinear), conjugate symmetry, positive-definite.
- Outer product is a map "ket × bra → operator".
- Operators on subsystems act as A⊗I or I⊗B on the full space.
- Unitaries are "generalised rotations": preserve inner products, hence probabilities.

##Worked Example

In [1]:
# ## 2. Worked examples (runnable)
# Helper utilities for bra-ket algebra and Kronecker products.

import numpy as np

def dagger(A): return A.conj().T
def inner(phi, psi):  # ⟨phi|psi⟩
    return np.vdot(phi, psi)  # conj(phi)·psi
def outer(psi, phi):  # |psi⟩⟨phi|
    return np.outer(psi, phi.conj())
def kron(*args):
    M = np.array([1.0], dtype=complex)
    for A in args:
        M = np.kron(M, A)
    return M
def normalise(psi):
    n = np.linalg.norm(psi)
    return psi/n if n>0 else psi

def is_unitary(U, tol=1e-10):
    I = np.eye(U.shape[0], dtype=complex)
    return np.allclose(dagger(U) @ U, I, atol=tol) and np.allclose(U @ dagger(U), I, atol=tol)

# One-qubit basis
ket0 = np.array([1,0], dtype=complex)
ket1 = np.array([0,1], dtype=complex)

# Pauli + Hadamard
X = np.array([[0,1],[1,0]], dtype=complex)
Y = np.array([[0,-1j],[1j,0]], dtype=complex)
Z = np.array([[1,0],[0,-1]], dtype=complex)
H = (1/np.sqrt(2))*np.array([[1,1],[1,-1]], dtype=complex)

X, Y, Z, H


(array([[0.+0.j, 1.+0.j],
        [1.+0.j, 0.+0.j]]),
 array([[ 0.+0.j, -0.-1.j],
        [ 0.+1.j,  0.+0.j]]),
 array([[ 1.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j]]),
 array([[ 0.70710678+0.j,  0.70710678+0.j],
        [ 0.70710678+0.j, -0.70710678+0.j]]))

In [2]:
# Example A — Tensor product basics: bases multiply, coefficients factor for product states
# Build |psi⟩⊗|phi⟩ and verify coefficients multiply.

# Single-qubit states
psi = normalise(2*ket0 + 1j*ket1)   # arbitrary
phi = normalise(ket0 - ket1)        # arbitrary

psi_phi = kron(psi, phi)            # |psi⟩⊗|phi⟩ in 4D
print("dim(H1)·dim(H2) =", 2*2, " ; dim(H1⊗H2) =", psi_phi.size)

# Coefficients multiply:
# |psi⟩ = [a0, a1], |phi⟩ = [b0, b1] → |psi⟩⊗|phi⟩ = [a0b0, a0b1, a1b0, a1b1]
a0,a1 = psi
b0,b1 = phi
expected = np.array([a0*b0, a0*b1, a1*b0, a1*b1])
print("Factor check:", np.allclose(psi_phi, expected))
psi_phi


dim(H1)·dim(H2) = 4  ; dim(H1⊗H2) = 4
Factor check: True


array([ 0.63245553+0.j        , -0.63245553+0.j        ,
        0.        +0.31622777j, -0.        -0.31622777j])

In [3]:
# Example B — Entanglement test via Schmidt rank (matrix rank of coefficients)
# Product states have rank 1; Bell states have rank 2.

# Build Bell state |Φ−⟩ = (|00⟩ − |11⟩)/√2
phi_minus = (kron(ket0,ket0) - kron(ket1,ket1))/np.sqrt(2)

# Reshape 4-vector to 2×2 "coefficient matrix" C_{ij}
C = phi_minus.reshape(2,2)
rank = np.linalg.matrix_rank(C)
print("Coefficient matrix:\n", C)
print("Schmidt rank =", rank, "→", "ENTANGLED" if rank>1 else "product")


Coefficient matrix:
 [[ 0.70710678+0.j  0.        +0.j]
 [ 0.        +0.j -0.70710678+0.j]]
Schmidt rank = 2 → ENTANGLED


In [4]:
# Example C — Inner product axioms + product rule
# Verify conjugate symmetry, right linearity, and ⟨a⊗b|c⊗d⟩ = ⟨a|c⟩⟨b|d⟩.

a = normalise(ket0 + 1j*ket1)
b = normalise(ket0 - 1j*ket1)
c = normalise(ket0 + ket1)
d = normalise(ket0 - ket1)

lhs = inner(kron(a,b), kron(c,d))
rhs = inner(a,c)*inner(b,d)

cs1 = inner(a,b)
cs2 = inner(b,a).conjugate()

alpha = 2-3j
lin_right = inner(a, alpha*b) - alpha*inner(a,b)

print("Product rule holds:", np.allclose(lhs, rhs))
print("Conjugate symmetry:", np.allclose(cs1, cs2))
print("Right linearity residual ~ 0:", abs(lin_right))


Product rule holds: True
Conjugate symmetry: True
Right linearity residual ~ 0: 0.0


In [5]:
# Example D — Outer products: projectors and density matrices
psi = normalise(ket0 + ket1)      # |+⟩
P = outer(psi, psi)               # projector |ψ⟩⟨ψ|
print("Projector idempotent? ||P^2 - P|| =", np.linalg.norm(P@P - P))

# Project arbitrary vector onto span{|ψ⟩}
v = normalise(2*ket0 - 1j*ket1)
proj_v = P @ v
print("Projection check: v' ∥ ψ:", np.allclose(np.cross(proj_v, psi), [0,0,0]))
P


Projector idempotent? ||P^2 - P|| = 2.220446049250313e-16
Projection check: v' ∥ ψ: True


/tmp/ipython-input-4055632548.py:9: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  print("Projection check: v' ∥ ψ:", np.allclose(np.cross(proj_v, psi), [0,0,0]))


array([[0.5+0.j, 0.5+0.j],
       [0.5+0.j, 0.5+0.j]])

In [6]:
# Example E — Operators on subsystems: A⊗I and I⊗B; CNOT; preservation of inner product
I2 = np.eye(2, dtype=complex)

# Apply X on the first qubit, Z on the second, to |10⟩
state = kron(ket1, ket0)
new_state = kron(X, I2) @ state         # X on qubit 0
new_state = kron(I2, Z) @ new_state     # then Z on qubit 1
print("|10⟩ →", new_state)

# Build CNOT (control = first qubit, target = second)
CNOT = np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,0,1],
    [0,0,1,0]
], dtype=complex)
print("CNOT unitary?", is_unitary(CNOT))

# Create Bell state via (H⊗I) then CNOT
bell = CNOT @ (kron(H, I2) @ kron(ket0, ket0))
print("Bell norm:", np.linalg.norm(bell))

# Inner product preservation by a unitary
v1 = normalise(np.random.randn(4) + 1j*np.random.randn(4))
v2 = normalise(np.random.randn(4) + 1j*np.random.randn(4))
lhs = inner(CNOT@v1, CNOT@v2)
rhs = inner(v1, v2)
print("⟨Uv1|Uv2⟩ == ⟨v1|v2⟩ ?", np.allclose(lhs, rhs))


|10⟩ → [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
CNOT unitary? True
Bell norm: 0.9999999999999999
⟨Uv1|Uv2⟩ == ⟨v1|v2⟩ ? True


In [7]:
# Example F — Unitary checks: U†U=I, eigenvalues on the unit circle
for name, U in [("X",X), ("Y",Y), ("Z",Z), ("H",H), ("CNOT",CNOT)]:
    evals = np.linalg.eigvals(U)
    mags = np.abs(evals)
    print(f"{name:5s} unitary? {is_unitary(U)} ; eigenvalue magnitudes: {np.round(mags,6)}")


X     unitary? True ; eigenvalue magnitudes: [1. 1.]
Y     unitary? True ; eigenvalue magnitudes: [1. 1.]
Z     unitary? True ; eigenvalue magnitudes: [1. 1.]
H     unitary? True ; eigenvalue magnitudes: [1. 1.]
CNOT  unitary? True ; eigenvalue magnitudes: [1. 1. 1. 1.]


## 3. Key equations and statements

- **Tensor product linearity & distributivity**
  $$
  (\alpha|\psi\rangle)\otimes|\phi\rangle = \alpha(|\psi\rangle\otimes|\phi\rangle),\quad
  (|\psi\rangle+|\psi'\rangle)\otimes|\phi\rangle=|\psi\rangle\otimes|\phi\rangle+|\psi'\rangle\otimes|\phi\rangle
  $$
- **Inner product axioms (complex)**
  1) Linearity in right slot: $\langle \chi|(\alpha|\psi\rangle+\beta|\phi\rangle)=\alpha\langle\chi|\psi\rangle+\beta\langle\chi|\phi\rangle$  
  2) Conjugate symmetry: $\langle \psi|\phi\rangle=\langle \phi|\psi\rangle^\*$  
  3) Positive-definite: $\langle \psi|\psi\rangle\ge 0$, with equality iff $|\psi\rangle=0$.
- **Product rule**  
  $\langle a\!\otimes\!b\,|\,c\!\otimes\!d\rangle=\langle a|c\rangle\,\langle b|d\rangle$.
- **Outer product / projector**  
  $P_\psi=|\psi\rangle\langle\psi|,\quad P_\psi^2=P_\psi,\quad P_\psi^\dagger=P_\psi.$
- **Operator tensoring**  
  $(A\otimes B)(|\psi\rangle\otimes|\phi\rangle)=(A|\psi\rangle)\otimes(B|\phi\rangle).$
- **Unitary**  
  $U^\dagger U=I\;\Rightarrow\;$ preserves inner products and norms; eigenvalues $e^{i\theta}$.


## 4. Product vs entangled (intuition)
- **Product state** $|\psi\rangle\otimes|\phi\rangle$: fully described by smaller states; reduced subsystems are **pure**.
- **Entangled state** (e.g. Bell): cannot be factorised; reduced subsystems are **mixed** ($\rho_A=\tfrac12 I$).  
  _Optional exercise below shows this via partial trace._


In [8]:
# OPTIONAL — Partial trace to show reduced mixedness for a Bell state
def partial_trace_rhoAB_to_A(rhoAB):
    # 2 qubits (2x2 each). Trace out B.
    rhoA = np.zeros((2,2), dtype=complex)
    for i in range(2):
        for j in range(2):
            # sum over B indices k
            for k in range(2):
                rhoA[i,j] += rhoAB[2*i+k, 2*j+k]
    return rhoA

rho_bell = outer(bell, bell)
rhoA = partial_trace_rhoAB_to_A(rho_bell)
print("ρ_A for Bell =\n", rhoA, "\nIs maximally mixed (≈ I/2)?",
      np.allclose(rhoA, 0.5*np.eye(2)))


ρ_A for Bell =
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]] 
Is maximally mixed (≈ I/2)? True


## 5. Try it yourself
1) **Schmidt test:** Generate random 2-qubit states; reshape to 2×2 and check rank. Which are entangled?  
2) **Operator on one qubit:** Build $U = Z\otimes X$; apply to each computational basis state; verify mapping.  
3) **Projector algebra:** Show that $P_\psi P_\phi = |\psi\rangle\langle\psi|\phi\rangle\langle\phi|$ and interpret the scalar.  
4) **Unitary eigenvalues:** Compute eigenvalues of a random $2\times2$ unitary (e.g., from QR decomposition) and confirm $|λ|=1$.  
5) **Inner product factorisation:** Prove (and test numerically) that $\langle a\!\otimes\!b\,|\,c\!\otimes\!d\rangle$ factorises.


## 6. Reflection
- In your own words, why does **entanglement** require the tensor product (and not just classical correlation)?  
- How do inner-product and unitarity guarantees translate into “probabilities sum to 1” during a computation?  
- Where do you expect $A\otimes I$ and $I\otimes B$ patterns to show up in algorithms you care about (e.g., QFT, oracles, variational layers)?


### Optional images to add
1) **Tensor-basis grid:** a 2×2 grid showing $|00\rangle,|01\rangle,|10\rangle,|11\rangle$ as $|u_i\rangle\otimes|v_j\rangle$.  
   _Caption:_ “Basis of $\mathcal H_1\otimes\mathcal H_2$ is the product of bases.”  
2) **Coefficient matrix heatmaps:** product state (rank-1 outer product) vs Bell state (rank-2) as $2\times2$ heatmaps.  
   _Caption:_ “Schmidt rank distinguishes product vs entangled.”  
3) **“Columns orthonormal” diagram:** unitary matrix with orthonormal columns (arrows on a circle).  
   _Caption:_ “Unitary columns are orthonormal; eigenvalues on the unit circle.”

---
### Links
- **Open in Colab (from GitHub):** replace `YOUR_GITHUB_USERNAME/qucode-21days`
  - `https://colab.research.google.com/github/YOUR_GITHUB_USERNAME/qucode-21days/blob/main/Day05_Multi‑Qubit_Systems_and_Tensor_Products.ipynb.ipynb`
- **Report an issue / suggest a fix:** link to your repo issues page
